# Import Library

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib as mpl
import seaborn as sns
import numpy as np 
import re
import os
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
from datetime import datetime, timedelta

# Visualization Style

In [ ]:
formatter = ticker.StrMethodFormatter('Rp.{x:,.0f}')

sns.set_theme(style='darkgrid')
sns.set_palette("deep")
mpl.rcParams['font.family'] = 'sans'
mpl.rcParams['font.size'] = 12
mpl.rcParams['axes.titlesize'] = 16
mpl.rcParams['axes.labelsize'] = 14
mpl.rcParams['xtick.labelsize'] = 10
mpl.rcParams['ytick.labelsize'] = 10

# Data Collecting

- Import system path

In [ ]:
import sys
sys.path.append('..')

- Import module

In [ ]:
from scripts import get_data_from_mongo

- Get Data from MongoDB

In [ ]:
df_import = get_data_from_mongo.get_data_mongo('tanah_bali_raw')
df_import = pd.DataFrame(df_import)

In [ ]:
df = df_import.copy()
df.head()

# Data Understanding

### Check Dataframe Dimension

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df.info()

# Data Preprocessing

### Handling Missing Value

In [ ]:
df.isna().sum().to_frame().T

### Handling Duplicate Data

- Drop _id Column

In [ ]:
df = df.drop(columns='_id', axis=1)
df.head()

- Check Duplicate Data

In [ ]:
dup_data = df.duplicated().sum()
print(f'Jumlah data duplicate: {dup_data}')
print(f'Dari total {df.shape[0]} data')

In [ ]:
df[df.duplicated()]

- Remove Duplicate Data

In [ ]:
df.drop_duplicates(keep='first', inplace=True)

dup_data = df.duplicated().sum()
print(f'Jumlah data duplicate: {dup_data}')
print(f'Dari total {df.shape[0]} data')

- Remove Unnessecary Data

In [ ]:
df = df.drop(columns='title', axis=1)
df.columns

### Cleaning Date Column

- Check Date Column

In [ ]:
df['date'].head().to_frame()

- Convert String to Datetime Data Type

In [ ]:
import locale
locale.setlocale(locale.LC_TIME, 'id_ID.UTF-8')

def convert_relative_data(description, timestamp):
    match = re.search(r'(\d+) (menit|jam|hari|minggu|bulan)', description)
    if not match:
        return None
    
    value = int(match.group(1))
    unit = match.group(2)
    
    if unit == 'menit':
        delta = timedelta(minutes=value)
    elif unit == 'jam':
        delta = timedelta(hours=value)
    elif unit == 'hari':
        delta = timedelta(days=value)
    elif unit == 'minggu':
        delta = timedelta(weeks=value)
    elif unit == 'bulan':
        delta = timedelta(days=30 * value)
    else:
        delta = timedelta()
    
    date = timestamp - delta
    return date

def convert_absolute_date(text):
    date_pattern = r'\b\d{2}\s\w+\s\d{4}\b'
    match = re.search(date_pattern, text)
    if match:
        return datetime.strptime(match.group(0), '%d %B %Y')
    return None

def convert_to_timestamp(row):
    description = row['date']
    timestamp = row['timestamp']
    
    if 'yang lalu' in description:
        return convert_relative_data(description, timestamp)
    else:
        return convert_absolute_date(description)

In [ ]:
df['converted_date'] = df.apply(convert_to_timestamp, axis=1).dt.strftime('%Y-%m-%d')

df_cleaned_date = df.copy()
df_cleaned_date.head()

In [ ]:
df_cleaned_date.sort_values(by='converted_date', ascending=True)

- Backup Dataframe

In [ ]:
df = df_cleaned_date.copy()
df.head()

- Detele Converted_Date and Timestamp Columns 

In [ ]:
df['date'] = df['converted_date']
df =df.drop(columns=['timestamp', 'converted_date'])

df.head()

### Cleaning Land Area Column

- Check Land Area Column

In [ ]:
df['land_area'].head()

- Convert String to Float Datatype

In [ ]:
df['numeric_land_area'] = df['land_area'].str.replace(r'm²', '', regex=True).astype(float)

df_cleaned_land_area = df.copy()

df.head()

- Backup Dataframe

In [ ]:
df = df_cleaned_land_area.copy()

df.head()

- Delete Numeric_Land_Area Column

In [ ]:
df['land_area'] = df['numeric_land_area']
df = df.drop(columns='numeric_land_area', axis=1)

df.head()

### Cleaning Price Column

- Check Price Column

In [ ]:
df['price'].head()

- Check Anomaly Value on Price

In [ ]:
dirty_data = df[df['price'].str.contains('Miliar /m²')]
dirty_data

- Delete Anomaly Value

In [ ]:
df = df.drop(dirty_data.index)
df.head()

In [ ]:
dirty_data = df[df['price'].str.contains('Miliar /m²')]
dirty_data.count()

- Change String to Float Datatype

In [ ]:
def parse_price(price_str):
    is_per_square_meter = "/m²" in price_str
    
    numeric_value = re.findall(r'[0-9,]+', price_str)
    
    if not numeric_value:
        return None
    
    numeric_value = float(numeric_value[0].replace(',', '.'))
    
    if "Triliun" in price_str:
        numeric_value *= 1_000_000_000_000
    elif "Miliar" in price_str:
        numeric_value *= 1_000_000_000
    elif "Juta" in price_str:
        numeric_value *= 1_000_000
    elif "Ribu" in price_str:
        numeric_value *= 1_000_000
    
    return is_per_square_meter, numeric_value

In [ ]:
df['is_per_square'], df['price_numeric'] = zip(*df['price'].apply(parse_price))

df['price_total'] = df.apply(lambda row: row['price_numeric'] * row['land_area'] if row['is_per_square'] else row['price_numeric'], axis=1).astype(float)
df['price_per_m2'] = df.apply(lambda row: row['price_numeric'] if row['is_per_square'] else row['price_total'] / row['land_area'], axis=1).astype(float)

df.head()

- Check Value with Readable Columns

In [ ]:
df['Price_Total_str'] = df['price_total'].apply(lambda x: f"{float(x):,.0f}")
df['Price_per_m2_str'] = df['price_per_m2'].apply(lambda x: f"{float(x):,.0f}")

df_cleaned_price = df.copy()

df.head()

- Backup Dataframe

In [ ]:
df = df_cleaned_price.copy()
df.head()

- Delete Unnecessary Columns

In [ ]:
df['price_per_m2'] = df['price_per_m2']
df['price_total'] = df['price_total']

df = df.drop(columns=['price', 'is_per_square', 'price_numeric', 'Price_Total_str', 'Price_per_m2_str'], axis=1)

df.head()


### Clean Address Column

- Check Address Column

In [ ]:
df['address'].head()

- Split Address to Subdistrict and Regency Column

In [ ]:
df[['subdistrict', 'regency']] = df['address'].str.split(',', n=1, expand=True)
df.head()

- Remove the withespace

In [ ]:
df['subdistrict'] = df['subdistrict'].str.strip()
df['regency'] = df['regency'].str.strip()
df.head()

- Check Unique Value of Regency Column

In [ ]:
df['regency'].unique()

- Check the Wrong Regency Value

In [ ]:
df[df['regency'] == 'Nusa Lembongan']

In [ ]:
df[df['regency'] == 'Semarapura']

- Change the Wrong Regency

In [ ]:
swap = (df['subdistrict'] == 'Klungkung') & (df['regency'] == 'Nusa Lembongan')

df.loc[swap, ['subdistrict', 'regency']] = ['Nusa Lembongan', 'Klungkung']
df['regency'] = df['regency'].replace('Semarapura', 'Klungkung')

df_cleaned_address = df.copy()
df['regency'].unique()

- Backup Dataframe

In [ ]:
df = df_cleaned_address.copy()
df = df.drop(columns='address', axis=1)

df.head()

### Cleaning Subdistrict Column

- Cleaning Subdistrict in Denpasar

In [ ]:
df[df['regency'] == 'Denpasar']['subdistrict'].unique()

In [ ]:
denpasar_utara_map = [
    "Dangin Puri Kaja",
    "Dangin Puri Kangin",
    "Dangin Puri Kauh",
    "Dauh Puri Kaja",
    "Peguyangan Kaja",
    "Peguyangan Kangin",
    "Pemecutan Kaja",
    "Ubung Kaja",
    "Peguyangan",
    "Tonja",
    "Ubung",
    "Gatot Subroto", 
    "Danginpuri Kangin", 
    "Dauhpuri Kaja", 
    "Danginpuri Kaja"
]

df.loc[(df['subdistrict'].isin(denpasar_utara_map)) & (df['regency'] == 'Denpasar'), 'subdistrict'] = 'Denpasar Utara'

In [ ]:
denpasar_selatan_map = [
    "Pemogan", 
    "Sanur Kaja", 
    "Sanur Kauh", 
    "Sidakarya", 
    "Panjer", 
    "Pedungan", 
    "Renon", 
    "Sanur", 
    "Serangan", 
    "Sesetan",
    "Pesanggaran", 
    "Suwung",
    "Gelogor Carik"

]

df.loc[(df['subdistrict'].isin(denpasar_selatan_map)) & (df['regency'] == 'Denpasar'), 'subdistrict'] = 'Denpasar Selatan'

In [ ]:
denpasar_timur_map = [
    "Dangin Puri Klod", 
    "Kesiman Kertalangu", 
    "Kesiman Petilan", 
    "Penatih Dangin Puri", 
    "Sumerta Kaja", 
    "Sumerta Kauh", 
    "Sumerta Klod/Kelod", 
    "Dangin Puri", 
    "Kesiman", 
    "Penatih", 
    "Sumerta",
    "Danginpuri Kelod", 
    "Penatihdanginpuri", 
    "Padanggalak", 
    "Sumerta Kelod",
    "Danginpuri"
]

df.loc[(df['subdistrict'].isin(denpasar_timur_map)) & (df['regency'] == 'Denpasar'), 'subdistrict'] = 'Denpasar Timur'

In [ ]:
denpasar_barat_map = [
    "Dauh Puri Kangin", 
    "Dauh Puri Kauh", 
    "Dauh Puri Klod", 
    "Padang Sambian Kaja", 
    "Padang Sambian Klod", 
    "Pemecutan Klod", 
    "Tegal Harum", 
    "Tegal Kerta", 
    "Dauh Puri", 
    "Padang Sambian", 
    "Pemecutan",
    "Mahendradata", 
    "Padangsambian Kaja", 
    "Padangsambian", 
    "Padangsambian Kelod", 
    "Pemecutan Kelod", 
    "Dauhpuri Kauh", 
    "Dauhpuri", 
    "Dauhpuri Kelod", 
    "Tegalkertha", 
    "Gunung soputan"
]


df.loc[(df['subdistrict'].isin(denpasar_barat_map)) & (df['regency'] == 'Denpasar'), 'subdistrict'] = 'Denpasar Barat'

In [ ]:
df[df['regency'] == 'Denpasar']['subdistrict'].unique()

In [ ]:
dalung_map = ['Dalung Permai']

df.loc[(df['subdistrict'].isin(dalung_map)) & (df['regency'] == 'Denpasar'), 'regency'] = 'Badung'

In [ ]:
df[df['regency'] == 'Denpasar']['subdistrict'].unique()

- Subdistrict in Badung Regency

In [ ]:
df[df['regency'] == 'Badung']['subdistrict'].unique()

In [ ]:
kuta_selatan_map = [
    "Pecatu",
    "Ungasan",
    "Kutuh",
    "Benoa",
    "Tanjung Benoa",
    "Jimbaran",
    "Nusa Dua", 
    "Goa Gong", 
    "Uluwatu", 
    "Balangan", 
    "Tundun Penyu", 
    "Mumbul", 
    "Puri Gading", 
    "Taman Griya",
    "Bukit"
]


df.loc[(df['subdistrict'].isin(kuta_selatan_map)) & (df['regency'] == 'Badung'), 'subdistrict'] = 'Kuta Selatan'

In [ ]:
kuta_map = [
    "Kedonganan",
    "Tuban",
    "Kuta",
    "Legian",
    "Seminyak",
    "Sunset Road", 
    "Dewi Sri",
    "Ngurah Rai",
    "Kartika Plaza"
]

df.loc[(df['subdistrict'].isin(kuta_map)) & (df['regency'] == 'Badung'), 'subdistrict'] = 'Kuta'

In [ ]:
kuta_utara_map = [
    "Canggu",
    "Dalung",
    "Tibubeneng",
    "Kerobokan",
    "Kerobokan Kelod",
    "Kerobokan Kaja",
    "Umalas",
    "Batu Belig",
    "Petitenget", 
    "Semer",
    "Gunung Salak",
    "Dalung Permai"
]

df.loc[(df['subdistrict'].isin(kuta_utara_map)) & (df['regency'] == 'Badung'), 'subdistrict'] = 'Kuta Utara'

In [ ]:
mengwi_map = [
    "Baha",
    "Buduk",
    "Cemagi",
    "Gulingan",
    "Kekeran",
    "Kuwum",
    "Mengwi",
    "Mengwitani",
    "Munggu",
    "Penarungan",
    "Pererenan",
    "Sembung",
    "Sobangan",
    "Tumbak",
    "Bayuh Werdi",
    "Bhuwana",
    "Abianbase",
    "Kapal",
    "Lukluk",
    "Sading",
    "Sempidi"
]

df.loc[(df['subdistrict'].isin(mengwi_map)) & (df['regency'] == 'Badung'), 'subdistrict'] = 'Mengwi'

In [ ]:
abiansemal_map = [
    "Abiansemal",
    "Angantaka",
    "Ayunan",
    "Blahkiuh",
    "Bongkasa",
    "Pertiwi",
    "Darmasaba",
    "Dauh Yeh Cani",
    "Jagapati",
    "Mambal",
    "Mekar Bhuana",
    "Punggul",
    "Sangeh",
    "Sedang",
    "Selat",
    "Sibang Gede",
    "Sibang Kaja",
    "Taman"
]

df.loc[(df['subdistrict'].isin(abiansemal_map)) & (df['regency'] == 'Badung'), 'subdistrict'] = 'Abiansemal'

In [ ]:
petang_map =[
    "Belok",
    "Carangsari",
    "Getasan",
    "Pangsan",
    "Pelaga",
    "Petang",
    "Sulangai"
]

df.loc[(df['subdistrict'].isin(petang_map)) & (df['regency'] == 'Badung'), 'subdistrict'] = 'Petang'

In [ ]:
blahbatuh_map = [
    "Bedulu",
    "Belega",
    "Blahbatuh",
    "Bona",
    "Buruan",
    "Keramas",
    "Medahan",
    "Pering",
    "Saba"
]

df.loc[(df['subdistrict'].isin(blahbatuh_map)) & (df['regency'] == 'Gianyar'), 'subdistrict'] = 'Blahbatuh'

In [ ]:
df[df['regency'] == 'Badung']['subdistrict'].unique()

In [ ]:
out_badung = ['Nyanyi',
              'Lombok',
              'Pesanggaran',
              'Klungkung']

drop_badung = df[(df['regency'] == 'Badung') & (df['subdistrict'].isin(out_badung))].index
df = df.drop(drop_badung)

In [ ]:
df[df['regency'] == 'Badung']['subdistrict'].unique()

- Subdistrict in Gianyar Regency

In [ ]:
df[df['regency'] == 'Gianyar']['subdistrict'].unique()

In [ ]:
gianyar_map = [
    "Bakbakan",
    "Lebih",
    "Petak",
    "Petak Kaja",
    "Serongga",
    "Siangan",
    "Sidan",
    "Sumita",
    "Suwat",
    "Tegal",
    "Tugu",
    "Temesi",
    "Tulikup",
    "Abianbase",
    "Beng",
    "Bitera",
    "Gianyar",
    "Samplangan",
    "Pantai Lebih",
    "Manggis"
]

df.loc[(df['subdistrict'].isin(gianyar_map)) & (df['regency'] == 'Gianyar'), 'subdistrict'] = 'Gianyar'

In [ ]:
payangan_map = [
    "Bresela",
    "Buahan",
    "Buahan Kaja",
    "Bukian",
    "Kelusa",
    "Kerta",
    "Melinggih",
    "Melinggih Kelod",
    "Puhu"
]

df.loc[(df['subdistrict'].isin(payangan_map)) & (df['regency'] == 'Gianyar'), 'subdistrict'] = 'Payangan'

In [ ]:
sukawati_map = [
    "Batuan",
    "Batuan Kaler",
    "Batubulan",
    "Batubulan Kangin",
    "Celuk",
    "Guwang",
    "Kemenuh",
    "Ketewel",
    "Singapadu",
    "Singapadu Kaler",
    "Singapadu Tengah",
    "Sukawati"
]

df.loc[(df['subdistrict'].isin(sukawati_map)) & (df['regency'] == 'Gianyar'), 'subdistrict'] = 'Sukawati'

In [ ]:
tampaksiring_map = [
    "Manukaya",
    "Pejeng",
    "Pejeng Kaja",
    "Pejeng Kangin",
    "Pejeng Kawan",
    "Pejeng Kelod",
    "Sanding",
    "Tampaksiring",
    "Pejengkawan"
]

df.loc[(df['subdistrict'].isin(tampaksiring_map)) & (df['regency'] == 'Gianyar'), 'subdistrict'] = 'Tampaksiring'

In [ ]:
tegallalang_map = [
    "Kedisan",
    "Keliki",
    "Kendran",
    "Pupuan",
    "Sebatu",
    "Taro",
    "Tegallalang",
    "Kenderan", 
    "Kedisah"
]

df.loc[(df['subdistrict'].isin(tegallalang_map)) & (df['regency'] == 'Gianyar'), 'subdistrict'] = 'Tegallalang'

In [ ]:
ubud_map = [
    "Kedewatan",
    "Lodtunduh",
    "Mas",
    "Peliatan",
    "Petulu",
    "Sayan",
    "Singakerta",
    "Ubud",
    "Sanggingan",
    "Singekerta"
]

df.loc[(df['subdistrict'].isin(ubud_map)) & (df['regency'] == 'Gianyar'), 'subdistrict'] = 'Ubud'

In [ ]:
blahbatuh_map = [
    "Belege", 
    "Cucukan"
]

df.loc[(df['subdistrict'].isin(blahbatuh_map)) & (df['regency'] == 'Gianyar'), 'subdistrict'] = 'Blahbatuh'

In [ ]:
df[df['regency'] == 'Gianyar']['subdistrict'].unique()

- Subdistrict in Bangli Regency

In [ ]:
bangli_map = [
    "Bunutin",
    "Kayubihi",
    "Landih",
    "Pengotan",
    "Taman Bali",
    "Bebalang",
    "Cempaga",
    "Kawan",
    "Kubu"
]

df.loc[(df['subdistrict'].isin(bangli_map)) & (df['regency'] == 'Bangli'), 'subdistrict'] = 'Bangli'

In [ ]:
kintamani_map = [
    "Abangsongan",
    "Abuan",
    "Awan",
    "Bantang",
    "Banua",
    "Batudinding",
    "Batukaang",
    "Batur Selatan",
    "Batur Tengah",
    "Batur Utara",
    "Bayungcerik",
    "Bayung Gede",
    "Belancan",
    "Belandingan",
    "Belanga",
    "Belantih",
    "Binyan",
    "Bonyoh",
    "Buahan",
    "Bunutin",
    "Catur",
    "Daup",
    "Dausa",
    "Gunungbau",
    "Katung",
    "Kedisan",
    "Kintamani",
    "Kutuh",
    "Langgahan",
    "Lembean",
    "Mangguh",
    "Manikliyu",
    "Mengani",
    "Pengejaran",
    "Pinggan",
    "Satra",
    "Sekaan",
    "Sekardadi",
    "Selulung",
    "Serai",
    "Siakin",
    "Songan A",
    "Songan B",
    "Subaya",
    "Sukawana",
    "Suter",
    "Terunyan",
    "Ulian"
]

df.loc[(df['subdistrict'].isin(kintamani_map)) & (df['regency'] == 'Bangli'), 'subdistrict'] = 'Kintamani'

In [ ]:
susut_map = [
    "Abuan",
    "Apuan",
    "Demulih",
    "Pengiangan",
    "Penglumbaran",
    "Selat",
    "Sulahan",
    "Susut",
    "Tiga"
]

df.loc[(df['subdistrict'].isin(susut_map)) & (df['regency'] == 'Bangli'), 'subdistrict'] = 'Susut'

In [ ]:
tembuku_map = [
    "Bangbang",
    "Jehem",
    "Peninjoan",
    "Tembuku",
    "Undisan",
    "Yangapi"
]

df.loc[(df['subdistrict'].isin(tembuku_map)) & (df['regency'] == 'Bangli'), 'subdistrict'] = 'Tembuku'

In [ ]:
df[df['regency'] == 'Bangli']['subdistrict'].unique()

- Subdistrict in Tabanan Regency

In [ ]:
df[df['regency'] == 'Tabanan']['subdistrict'].unique()

In [ ]:
baturiti_map = [
    "Angseri",
    "Antapan",
    "Apuan",
    "Bangli",
    "Batunya",
    "Baturiti",
    "Candikuning",
    "Luwus",
    "Mekarsari",
    "Perean",
    "Perean Kangin",
    "Perean Tengah",
    "Bedugul"
]

df.loc[(df['subdistrict'].isin(baturiti_map)) & (df['regency'] == 'Tabanan'), 'subdistrict'] = 'Baturiti'

In [ ]:
kediri_map = [
    "Abian Tuwung",
    "Banjar Anyar",
    "Belalang",
    "Bengkel",
    "Beraban",
    "Buwit",
    "Cepaka",
    "Kaba-kaba",
    "Kediri",
    "Nyambu",
    "Nyitdah",
    "Pandak Bandung",
    "Pandak Gede",
    "Pangkung Tibah",
    "Pejaten",
    "Tanah Lot",
    "Abiantuwung",
    "Kaba kaba",
    "Nyambung",
    "Banjaranyar",
    "Pangkutibah",
    "Pandakbandung",
    "Pandakgede"

]

df.loc[(df['subdistrict'].isin(kediri_map)) & (df['regency'] == 'Tabanan'), 'subdistrict'] = 'Kediri'


In [ ]:
kerambitan_map = [
    "Batuaji",
    "Baturiti",
    "Belumbang",
    "Kelating",
    "Kerambitan",
    "Kesiut",
    "Kukuh",
    "Meliling",
    "Pangkung Karung",
    "Penarukan",
    "Samsam",
    "Sembung Gede",
    "Tibubiu",
    "Timpag",
    "Tista",
    "Tibubiyu",
    "Sembunggede",
    "Pangkungkarung"

]

df.loc[(df['subdistrict'].isin(kerambitan_map)) & (df['regency'] == 'Tabanan'), 'subdistrict'] = 'Kerambitan'


In [ ]:
marga_map = [
    "Baru",
    "Batannyuh",
    "Beringkit",
    "Cau Belayu",
    "Geluntung",
    "Kukuh",
    "Kuwum",
    "Marga",
    "Marga Dajan Puri",
    "Marga Dauh Puri",
    "Payangan",
    "Peken",
    "Petiga",
    "Selanbawak",
    "Tegaljadi",
    "Tua",
    "Caubelayu"
]

df.loc[(df['subdistrict'].isin(marga_map)) & (df['regency'] == 'Tabanan'), 'subdistrict'] = 'Marga'

In [ ]:
penebel_map = [
    "Babahan",
    "Biaung",
    "Buruan",
    "Jatiluwih",
    "Jegu",
    "Mengesta",
    "Penatahan",
    "Penebel",
    "Pesagi",
    "Pitra",
    "Rejasa",
    "Rianggede",
    "Sangketan",
    "Senganan",
    "Tajen",
    "Tegallinggah",
    "Tengkudak",
    "Wongaya Gede",
    "Soka",
    "Wongayagede"
]

df.loc[(df['subdistrict'].isin(penebel_map)) & (df['regency'] == 'Tabanan'), 'subdistrict'] = 'Penebel'

In [ ]:
pupuan_map = [
    "Bantiran",
    "Batungsel",
    "Belatungan",
    "Belimbing",
    "Jelijih Punggang",
    "Karya Sari",
    "Kebon Padangan",
    "Munduk Temu",
    "Padangan",
    "Pajahan",
    "Pujungan",
    "Pupuan",
    "Sai",
    "Sanda",
    "Munduktemu",
    "Kebonpadangan"
]

df.loc[(df['subdistrict'].isin(pupuan_map)) & (df['regency'] == 'Tabanan'), 'subdistrict'] = 'Pupuan'

In [ ]:
selemadeg_map = [
    "Antap",
    "Bajera",
    "Bajera Utara",
    "Berembeng",
    "Manikyang",
    "Pupuan Sawah",
    "Selemadeg",
    "Serampingan",
    "Wanagiri",
    "Wanagiri Kauh",
    "Pupuansawah"
]

df.loc[(df['subdistrict'].isin(selemadeg_map)) & (df['regency'] == 'Tabanan'), 'subdistrict'] = 'Selemadeg'

In [ ]:
selemadeg_barat_map = [
    "Angkah",
    "Antosari",
    "Bengkel Sari",
    "Lalang Linggah",
    "Lumbung",
    "Lumbung Kauh",
    "Mundeh",
    "Mundeh Kangin",
    "Mundeh Kauh",
    "Selabih",
    "Tiying Gading",
    "Lalanglinggah",
    "Yeh Leh"
]

df.loc[(df['subdistrict'].isin(selemadeg_barat_map)) & (df['regency'] == 'Tabanan'), 'subdistrict'] = 'Selemadeg Barat'

In [ ]:
selemadeg_timur_map = [
    "Bantas",
    "Beraban",
    "Dalang",
    "Gadungan",
    "Gadung Sari",
    "Gunung Salak",
    "Mambang",
    "Megati",
    "Tangguntiti",
    "Tegal Mengkeb",
    "Tegalmengkeb",
    "Klecung",
    "Gunungsalak"

]

df.loc[(df['subdistrict'].isin(selemadeg_timur_map)) & (df['regency'] == 'Tabanan'), 'subdistrict'] = 'Selemadeg Timur'

In [ ]:
tabanan_map = [
    "Bongan",
    "Buahan",
    "Dajan Peken",
    "Dauh Peken",
    "Delod Peken",
    "Denbantas",
    "Gubug",
    "Sesandan",
    "Subamia",
    "Sudimara",
    "Tunjuk",
    "Wanasari",
    "Delodpeken",
    "Dauhpeken",
    "Dajanpeken"

]

df.loc[(df['subdistrict'].isin(tabanan_map)) & (df['regency'] == 'Tabanan'), 'subdistrict'] = 'Tabanan'

In [ ]:
df[df['regency'] == 'Tabanan']['subdistrict'].unique()

- Subdistrict in Karangasem Regency

In [ ]:
df[df['regency'] == 'Karangasem']['subdistrict'].unique()

In [ ]:
karangasem_map = [
    "Bugbug",
    "Bukit",
    "Pertima",
    "Seraya Barat",
    "Seraya Tengah",
    "Seraya Timur",
    "Tegallinggah",
    "Tumbu",
    "Subagan",
    "Padang Kerta",
    "Karangasem",
    "Seraya"
]

df.loc[(df['subdistrict'].isin(karangasem_map)) & (df['regency'] == 'Karangasem'), 'subdistrict'] = 'Karangasem'

In [ ]:
abang_map = [
    "Ababi",
    "Abang",
    "Bunutan",
    "Culik",
    "Datah",
    "Kerta Mandala",
    "Kesimpar",
    "Labasari",
    "Nawa Kerthi",
    "Pidpid",
    "Purwakerti",
    "Tista",
    "Tiyingtali",
    "Tri Bhuana",
    "Amed"
]

df.loc[(df['subdistrict'].isin(abang_map)) & (df['regency'] == 'Karangasem'), 'subdistrict'] = 'Abang'

In [ ]:
kubu_map = [
    "Ban",
    "Baturinggit",
    "Dukuh",
    "Kubu",
    "Sukadana",
    "Tianyar",
    "Tianyar Barat",
    "Tianyar Tengah",
    "Tulamben"
]

df.loc[(df['subdistrict'].isin(kubu_map)) & (df['regency'] == 'Karangasem'), 'subdistrict'] = 'Kubu'

In [ ]:
manggis_map = [
    "Antiga",
    "Antiga Kelod",
    "Gegelang",
    "Manggis",
    "Ngis",
    "Nyuhtebel",
    "Padangbai",
    "Persedahan",
    "Selumbung",
    "Sengkidu",
    "Tenganan",
    "Ulakan",
    "Candi Dasa"
]

df.loc[(df['subdistrict'].isin(manggis_map)) & (df['regency'] == 'Karangasem'), 'subdistrict'] = 'Manggis'

In [ ]:
rendang_map = [
    "Besakih",
    "Menanga",
    "Nongan",
    "Pempatan",
    "Pesaban",
    "Rendang"
]

df.loc[(df['subdistrict'].isin(rendang_map)) & (df['regency'] == 'Karangasem'), 'subdistrict'] = 'Rendang'

In [ ]:
bebandem_map = [
    "Bebandem",
    "Buana Giri",
    "Budakeling",
    "Bungaya",
    "Bungaya Kangin",
    "Jungutan",
    "Macang",
    "Sibetan"
]

df.loc[(df['subdistrict'].isin(bebandem_map)) & (df['regency'] == 'Karangasem'), 'subdistrict'] = 'Bebandem'

In [ ]:
sidemen_map = [
    "Kertha Buana",
    "Lokasari",
    "Sangkan Gunung",
    "Sidemen",
    "Sindu Wati",
    "Talibeng",
    "Tangkup",
    "Telaga Tawang",
    "Tri Eka Buana",
    "Wisma Kerta"
]

df.loc[(df['subdistrict'].isin(sidemen_map)) & (df['regency'] == 'Karangasem'), 'subdistrict'] = 'Sidemen'

In [ ]:
selat_map = [
    "Amertha Buana",
    "Duda",
    "Duda Timur",
    "Duda Utara",
    "Muncan",
    "Pering Sari",
    "Sebudi",
    "Selat"
]

df.loc[(df['subdistrict'].isin(selat_map)) & (df['regency'] == 'Karangasem'), 'subdistrict'] = 'Selat'

In [ ]:
df[df['regency'] == 'Karangasem']['subdistrict'].unique()

- Subdistrict in Klungkung Regency

In [ ]:
df[df['regency'] == 'Klungkung']['subdistrict'].unique()

In [ ]:
banjarangkan_map = [
    "Aan",
    "Bakas",
    "Banjarangkan",
    "Bungbungan",
    "Getakan",
    "Negari",
    "Nyalian",
    "Nyanglan",
    "Takmung",
    "Tihingan",
    "Timuhun",
    "Tohpati",
    "Tusan",
    "Banjarankan"
]

df.loc[(df['subdistrict'].isin(banjarangkan_map)) & (df['regency'] == 'Klungkung'), 'subdistrict'] = 'Banjarangkan'

In [ ]:
klungkung_map = [
    "Akah",
    "Gelgel",
    "Jumpai",
    "Kamasan",
    "Kampung Gelgel",
    "Manduang",
    "Satra",
    "Selat",
    "Selisihan",
    "Tangkas",
    "Tegak",
    "Tojan",
    "Semarapura Kaja",
    "Semarapura Kangin",
    "Semarapura Kauh",
    "Semarapura Klod",
    "Semarapura Klod Kangin",
    "Semarapura Tengah",
    "Klungkung Kota"
]

df.loc[(df['subdistrict'].isin(klungkung_map)) & (df['regency'] == 'Klungkung'), 'subdistrict'] = 'Klungkung'

In [ ]:
dawan_map = [
    "Besan",
    "Dawan Kaler",
    "Dawan Klod",
    "Gunaksa",
    "Kampung Kusamba",
    "Kusamba",
    "Paksebali",
    "Pesinggahan",
    "Pikat",
    "Sampalan Klod",
    "Sampalan Tengah",
    "Sulang",
    "Dawan Kelod"
]

df.loc[(df['subdistrict'].isin(dawan_map)) & (df['regency'] == 'Klungkung'), 'subdistrict'] = 'Dawan'

In [ ]:
nusa_penida_map = [
    "Batukandik",
    "Batumadeg",
    "Batununggul",
    "Bunga Mekar",
    "Jungutbatu",
    "Kampung Toyapakeh",
    "Klumpu",
    "Kutampi",
    "Kutampi Kaler",
    "Lembongan",
    "Ped",
    "Pejukutan",
    "Sakti",
    "Sekartaji",
    "Suana",
    "Tanglad",
    "Nusa Lembongan", 
    "Klumbu"
]

df.loc[(df['subdistrict'].isin(nusa_penida_map)) & (df['regency'] == 'Klungkung'), 'subdistrict'] = 'Nusa Penida'

In [ ]:
df[df['regency'] == 'Klungkung']['subdistrict'].unique()

In [ ]:
out_klungkung = ['Sauna']

drop_klungkung = df[(df['regency'] == 'Klungkung') & (df['subdistrict'].isin(out_klungkung))].index
df = df.drop(drop_klungkung)

In [ ]:
df[df['regency'] == 'Klungkung']['subdistrict'].unique()

- Subdistrict in Jembrana Regency

In [ ]:
df[df['regency'] == 'Jembrana']['subdistrict'].unique()

In [ ]:
melaya_map = [
    "Blimbing Sari",
    "Candikusuma",
    "Ekasari",
    "Manistutu",
    "Melaya",
    "Nusasari",
    "Tukadaya",
    "Tuwed",
    "Warnasari",
    "Gilimanuk",
    "Candikesuma"
]

df.loc[(df['subdistrict'].isin(melaya_map)) & (df['regency'] == 'Jembrana'), 'subdistrict'] = 'Melaya'

In [ ]:
negara_map = [
    "Baluk",
    "Banyubiru",
    "Berangbang",
    "Cupel",
    "Kaliakah",
    "Pengambengan",
    "Tegal Badeng Barat",
    "Tegal Badeng Timur",
    "Baler Bale Agung",
    "Banjar Tengah",
    "Lelateng",
    "Loloan Barat",
    "Tegalbadeng Timur",
    "Balerbaleagung", 
    "Pangambengan"
]

df.loc[(df['subdistrict'].isin(negara_map)) & (df['regency'] == 'Jembrana'), 'subdistrict'] = 'Negara'

In [ ]:
jembrana_map = [
    "Air Kuning",
    "Batuagung",
    "Budeng",
    "Dangintukadaya",
    "Perancak",
    "Yeh Kuning",
    "Dauhwaru",
    "Loloan Timur",
    "Pendem",
    "Sangkaragung",
    "Airkuning"
]

df.loc[(df['subdistrict'].isin(jembrana_map)) & (df['regency'] == 'Jembrana'), 'subdistrict'] = 'Jembrana'

In [ ]:
mendoyo_map = [
    "Delod Berawah",
    "Mendoyo Dangin Tukad",
    "Mendoyo Dauh Tukad",
    "Penyaringan",
    "Pergung",
    "Pohsanten",
    "Yeh Embang",
    "Yeh Embang Kangin",
    "Yeh Embang Kauh",
    "Yeh Sumbul",
    "Tegal Cangkring",
    "Yehembang Kangin",
    "Yehsumbul",
    "Mendoyo Dangintukad",
    "Delodberawah",
    "Yehembang",
    "Mendoyo Dauhtukad"
]

df.loc[(df['subdistrict'].isin(mendoyo_map)) & (df['regency'] == 'Jembrana'), 'subdistrict'] = 'Mendoyo'

In [ ]:
pekutatan_map = [
    "Asahduren",
    "Gumbrih",
    "Manggissari",
    "Medewi",
    "Pangyangan",
    "Pekutatan",
    "Pengragoan",
    "Pulukan",
    "Pangeragoan"
]

df.loc[(df['subdistrict'].isin(pekutatan_map)) & (df['regency'] == 'Jembrana'), 'subdistrict'] = 'Pekutatan'

In [ ]:
df[df['regency'] == 'Jembrana']['subdistrict'].unique()

- Subdistrict in Buleleng Regency

In [ ]:
df[df['regency'] == 'Buleleng']['subdistrict'].unique()

In [ ]:
gerokgak_map = [
    "Banyupoh",
    "Celukanbawang",
    "Gerokgak",
    "Musi",
    "Patas",
    "Pejarakan",
    "Pemuteran",
    "Pengulon",
    "Penyabangan",
    "Sanggalangit",
    "Sumberklampok",
    "Sumberklima",
    "Tinga-Tinga",
    "Tukadsumaga",
    "Tinga tinga",
    "Sumberkimia"
]

df.loc[(df['subdistrict'].isin(gerokgak_map)) & (df['regency'] == 'Buleleng'), 'subdistrict'] = 'Gerokgak'

In [ ]:
seririt_map = [
    "Banjar Asem",
    "Bestala",
    "Bubunan",
    "Gunungsari",
    "Joanyar",
    "Kalianget",
    "Kalisada",
    "Lokapaksa",
    "Mayong",
    "Munduk Bestala",
    "Pangkung Paruk",
    "Patemon",
    "Pengastulan",
    "Rangdu",
    "Ringdikit",
    "Sulanyah",
    "Tangguwisia",
    "Ularan",
    "Umeanyar",
    "Unggahan",
    "Banjarasem", "Pangkungparuk"
]

df.loc[(df['subdistrict'].isin(seririt_map)) & (df['regency'] == 'Buleleng'), 'subdistrict'] = 'Seririt'

In [ ]:
busungbiu_map = [
    "Bengkel",
    "Bongancina",
    "Busung Biu",
    "Kedis",
    "Kekeran",
    "Pelapuan",
    "Pucaksari",
    "Sepang",
    "Sepang Kelod",
    "Subuk",
    "Telaga",
    "Tinggarsari",
    "Tista",
    "Titab",
    "Umejero",
    "Puncaksari",
    "BusungBiu",
    "Busungbiu"
]

df.loc[(df['subdistrict'].isin(busungbiu_map)) & (df['regency'] == 'Buleleng'), 'subdistrict'] = 'Busung Biu'

In [ ]:
banjar_map = [
    "Banjar",
    "Banjar Tegeha",
    "Banyuatis",
    "Banyuseri",
    "Cempaga",
    "Dencarik",
    "Gesing",
    "Gobleg",
    "Kaliasem",
    "Kayuputih",
    "Munduk",
    "Pedawa",
    "Sidetapa",
    "Tampekan",
    "Temukus",
    "Tigawasa",
    "Tirtasari"
]

df.loc[(df['subdistrict'].isin(banjar_map)) & (df['regency'] == 'Buleleng'), 'subdistrict'] = 'Banjar'

In [ ]:
sukasada_map = [
    "Ambengan",
    "Git Git",
    "Kayu Putih",
    "Padang Bulia",
    "Pancasari",
    "Panji",
    "Panji Anom",
    "Pegadungan",
    "Pegayaman",
    "Sambangan",
    "Selat",
    "Silangjana",
    "Tegal Linggah",
    "Wanagiri",
    "Panjianom", 
    "Gitgit", 
]

df.loc[(df['subdistrict'].isin(sukasada_map)) & (df['regency'] == 'Buleleng'), 'subdistrict'] = 'Sukasada'

In [ ]:
buleleng_map = [
    "Alasangker",
    "Anturan",
    "Bakti Seraga",
    "Jinengdalem",
    "Kalibukbuk",
    "Nagasepaha",
    "Pemaron",
    "Penglatan",
    "Petandakan",
    "Poh Bergong",
    "Sari Mekar",
    "Tukadmungga",
    "Astina",
    "Banjar Bali",
    "Banjar Jawa",
    "Banjar Tegal",
    "Banyuasri",
    "Banyuning",
    "Beratan",
    "Kaliuntu",
    "Kampung Anyar",
    "Kampung Baru",
    "Kampung Bugis",
    "Kampung Kajanan",
    "Kampung Singaraja",
    "Kendran",
    "Liligundi",
    "Paket Agung",
    "Penarukan",
    "Lovina",
    "Singaraja",
    "Banjarbali",
    "Baktiseraga", 
    "Kalibubuk", 
]

df.loc[(df['subdistrict'].isin(buleleng_map)) & (df['regency'] == 'Buleleng'), 'subdistrict'] = 'Buleleng'

In [ ]:
sawan_map = [
    "Bebetin",
    "Bungkulan",
    "Galungan",
    "Giri Emas",
    "Jagaraga",
    "Kerobokan",
    "Lemukih",
    "Menyali",
    "Sangsit",
    "Sawan",
    "Sekumpul",
    "Sinabun",
    "Sudaji",
    "Suwug"
]

df.loc[(df['subdistrict'].isin(sawan_map)) & (df['regency'] == 'Buleleng'), 'subdistrict'] = 'Sawan'

In [ ]:
kubutambahan_map = [
    "Bengkala",
    "Bila",
    "Bontihing",
    "Bukti",
    "Bulian",
    "Depeha",
    "Kubutambahan",
    "Mengening",
    "Pakisan",
    "Tajun",
    "Tambakan",
    "Tamblang",
    "Tunjung",
    "Kubu Tambahan"
]

df.loc[(df['subdistrict'].isin(kubutambahan_map)) & (df['regency'] == 'Buleleng'), 'subdistrict'] = 'Kubu Tambahan'

In [ ]:
tejakula_map = [
    "Bondalem",
    "Julah",
    "Les",
    "Madenan",
    "Pacung",
    "Penuktukan",
    "Sambirenteng",
    "Sembiran",
    "Tejakula",
    "Tembok"
]

df.loc[(df['subdistrict'].isin(tejakula_map)) & (df['regency'] == 'Buleleng'), 'subdistrict'] = 'Tejakula'

In [ ]:
df[df['regency'] == 'Buleleng']['subdistrict'].unique()

- Check the Regency

In [ ]:
df['regency'].unique()

- Backup Dataframe

In [ ]:
df_new_subd = df.copy()

In [ ]:
df = df_new_subd.copy()
df.head()

# Noise Data Handling

### Noise Data on Land Area

In [ ]:
land_area = df['land_area']

plt.figure(figsize=(10, 6))
plt.hist(land_area, bins=30, edgecolor='k', alpha=0.7)
plt.title('Distribution of Land Area')
plt.xlabel('Land Area')
plt.ylabel('Frequency')
plt.yscale('log')
plt.xscale('log')
plt.show()

- Quantile Method to Handle Noise Data

In [ ]:
threshold = land_area.quantile(0.99)
filtered_land_area = df[land_area < threshold]

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(filtered_land_area['land_area'], bins=30, edgecolor='k', alpha=0.7)
plt.title('Distribution of Land Area (Filtered)')
plt.xlabel('Land Area')
plt.ylabel('Frequency')
plt.yscale('log')
plt.xscale('log')
plt.show()

- Backup Dataframe

In [ ]:
df = filtered_land_area.copy()

### Noise Data on Price Total

In [ ]:
price_total = df['price_total']

plt.figure(figsize=(10, 6))
plt.hist(price_total, bins=30, edgecolor='k', alpha=0.7)
plt.title('Distribution of Price Total')
plt.xlabel('Land Area')
plt.ylabel('Frequency')
plt.yscale('log')
plt.xscale('log')
plt.show()

- Quantile Method to Handle Noise Data

In [ ]:
threshold = price_total.quantile(0.99)
filtered_land_area = df[price_total < threshold]

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(filtered_land_area['price_total'], bins=30, edgecolor='k', alpha=0.7)
plt.title('Distribution of Price Total (Filtered)')
plt.xlabel('Price Total')
plt.ylabel('Frequency')
plt.yscale('log')
plt.xscale('log')
plt.show()

- Backup Dataframe

In [ ]:
df = filtered_land_area.copy()

### Noise Data on Price Per m2

In [ ]:
price_per_m2 = df['price_per_m2']

plt.figure(figsize=(10, 6))
plt.hist(price_per_m2, bins=30, edgecolor='k', alpha=0.7)
plt.title('Distribution of Price Per m2')
plt.xlabel('Price Per m2')
plt.ylabel('Frequency')
plt.yscale('log')
plt.xscale('log')
plt.show()

- Quantile Method to Handle Noise Data

In [ ]:
threshold = price_per_m2.quantile(0.99)
filtered_land_area = df[price_per_m2 < threshold]

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(filtered_land_area['price_per_m2'], bins=30, edgecolor='k', alpha=0.7)
plt.title('Distribution of Price Per m2 (Filtered)')
plt.xlabel('Price Per m2')
plt.ylabel('Frequency')
plt.yscale('log')
plt.xscale('log')
plt.show()

- Backup Dataframe

In [ ]:
df = filtered_land_area.copy()

# Outlier Handling

- Backup Dataframe

In [ ]:
df_outlier = df.copy()

In [ ]:
df = df_outlier.copy()

### Outlier Handling on Land Area Column

- Data Before Outlier Handling

In [ ]:
df['land_area'].describe()

In [ ]:
plt.figure(figsize=(10, 2))
sns.boxplot(x=df['land_area'])
plt.title('Boxplot of Land Area')
plt.xlabel('Land Area')
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df, x=df.index, y='land_area', alpha=0.5)
plt.title('Scatter Plot of Land Area')
plt.xlabel('Index')
plt.ylabel('Land Area ')
plt.grid(True)
plt.tight_layout()
plt.show()

- Make Log Function for Land Area

In [ ]:
df['log_land_area'] = np.log(df['land_area'])
df.head()

In [ ]:
plt.figure(figsize=(10, 2))
sns.boxplot(x=df['log_land_area'])
plt.title('Boxplot of Land Area')
plt.xlabel('Land Area')
plt.tight_layout()
plt.show()

- Outlier Handling with IQR Method

In [ ]:
Q1 = df['log_land_area'].quantile(0.25)
Q3 = df['log_land_area'].quantile(0.75)

# Menghitung Interquartile Range (IQR)
IQR = Q3 - Q1

# Menentukan ambang batas minimum untuk gaji
min_threshold = 0

# Menghitung batas bawah, dan memastikannya tidak negatif
lower_bound = max(Q1 - 1.5 * IQR, min_threshold)

# Menghitung batas atas
upper_bound = Q3 + 1.5 * IQR

print('lower_bound: ',lower_bound)
print('upper_bound: ', upper_bound)

In [ ]:
df = df[(df['log_land_area'] >= lower_bound) & (df['log_land_area'] <= upper_bound)]

- Data After Outlier Handling

In [ ]:
df['log_land_area'].describe()

In [ ]:
plt.figure(figsize=(15, 2))
sns.boxplot(x=df['log_land_area'])
plt.title('Boxplot of Land Area')
plt.xlabel('Land Area')
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df, x=df.index, y='log_land_area', alpha=0.5)
plt.title('Scatter Plot of Land Area')
plt.xlabel('Index')
plt.ylabel('Land Area')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
df.info()

### Outlier Handling of Price Total Column

- Data Before Outlier Handling

In [ ]:
df['price_total'].describe()

In [ ]:
plt.figure(figsize=(10, 2))
sns.boxplot(x=df['price_total'])
plt.title('Boxplot Price Total')
plt.xlabel('Price Total')
plt.gca().xaxis.set_major_formatter(formatter)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df, x=df.index, y='price_total', alpha=0.5)
plt.title('Scatter Plot of Total Price')
plt.xlabel('Index')
plt.ylabel('Total Price')
plt.grid(True)
plt.gca().yaxis.set_major_formatter(formatter)
plt.tight_layout()
plt.show()

- Make Log Function for Price Total

In [ ]:
df['log_price_total'] = np.log(df['price_total'])
df.head()

In [ ]:
plt.figure(figsize=(10, 2))
sns.boxplot(x=df['log_price_total'])
plt.title('Boxplot of Price Total')
plt.xlabel('Price Total')
plt.tight_layout()
plt.show()

- Outlier Handling with IQR Method

In [ ]:
Q1 = df['log_price_total'].quantile(0.25)
Q3 = df['log_price_total'].quantile(0.75)

# Menghitung Interquartile Range (IQR)
IQR = Q3 - Q1

# Menentukan ambang batas minimum untuk gaji
min_threshold = 0

# Menghitung batas bawah, dan memastikannya tidak negatif
lower_bound = max(Q1 - 1.5 * IQR, min_threshold)

# Menghitung batas atas
upper_bound = Q3 + 1.5 * IQR

print('lower_bound: ',lower_bound)
print('upper_bound: ', upper_bound)

In [ ]:
df = df[(df['log_price_total'] >= lower_bound) & (df['log_price_total'] <= upper_bound)]

- Data After Outlier Handling

In [ ]:
df['log_price_total'].describe()

In [ ]:
plt.figure(figsize=(15, 2))
sns.boxplot(x=df['log_price_total'])
plt.title('Boxplot of Price Total')
plt.xlabel('Price Total')
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df, x=df.index, y='log_price_total', alpha=0.5)
plt.title('Scatter Plot of Total Price')
plt.xlabel('Index')
plt.ylabel('Total Price')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
df.info()

### Outlier Handling of Price Per m2

- Data Before Outlier Handling

In [ ]:
df['price_per_m2'].describe()

In [ ]:
plt.figure(figsize=(10, 2))
sns.boxplot(x=df['price_per_m2'])
plt.title('Boxplot of Price per m2')
plt.xlabel('Price per m2')
plt.gca().xaxis.set_major_formatter(formatter)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df, x=df.index, y='price_per_m2', alpha=0.5)
plt.title('Scatter Plot of Price per m2')
plt.xlabel('Index')
plt.ylabel('Price per m2')
plt.grid(True)
plt.gca().yaxis.set_major_formatter(formatter)
plt.tight_layout()
plt.show()

- Log Function for Price Per m2

In [ ]:
df['log_price_per_m2'] = np.log(df['price_per_m2'])
df.head()

In [ ]:
plt.figure(figsize=(10, 2))
sns.boxplot(x=df['log_price_per_m2'])
plt.title('Boxplot of Price per m2')
plt.xlabel('Price per m2')
plt.tight_layout()
plt.show()

- Outlier Handling with IQR Method

In [ ]:
Q1 = df['log_price_per_m2'].quantile(0.25)
Q3 = df['log_price_per_m2'].quantile(0.75)

# Menghitung Interquartile Range (IQR)
IQR = Q3 - Q1

# Menentukan ambang batas minimum untuk gaji
min_threshold = 0

# Menghitung batas bawah, dan memastikannya tidak negatif
lower_bound = max(Q1 - 1.5 * IQR, min_threshold)

# Menghitung batas atas
upper_bound = Q3 + 1.5 * IQR

print('lower_bound: ',lower_bound)
print('upper_bound: ', upper_bound)

In [ ]:
df = df[(df['log_price_per_m2'] >= lower_bound) & (df['log_price_per_m2'] <= upper_bound)]

In [ ]:
df['log_price_per_m2'].describe()

In [ ]:
plt.figure(figsize=(10, 2))
sns.boxplot(x=df['log_price_per_m2'])
plt.title('Boxplot of Price per m2')
plt.xlabel('Price per m2')
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df, x=df.index, y='log_price_per_m2', alpha=0.5)
plt.title('Scatter Plot of Price per m2')
plt.xlabel('Index')
plt.ylabel('Price per m2')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
df.info()

In [ ]:
df['log_price_per_m2'].describe()

In [ ]:
df.head()

In [ ]:
filtered_df = df[(df['log_land_area'] > 0) & (df['log_price_total'] > 0) & (df['price_per_m2'] > 0)]
df = filtered_df

df.head()

# Data Transformation

### Convert Date Column to Datetime Datatype

In [ ]:
df['date'] = pd.to_datetime(df['date'])

df_date_conv = df.copy()
df_date_conv.head()

In [ ]:
df_date_conv.info()

- Backup Dataframe

In [ ]:
df = df_date_conv.copy()
df.head()

# Feature Engineering

### Check Dataframe

In [ ]:
df.head()

### Check Land Area Column

In [ ]:
df['land_area'].describe()

### Make New Feature Based on Land Area

In [ ]:
bins = [0, 400, 2770, float('inf')]
labels = ['small', 'medium', 'large']
df['land_area_cat'] = pd.cut(df['land_area'], bins=bins, labels=labels)

df.sort_values(by='land_area', ascending=True)

# Export Dataframe to MongoDB

- Import Module

In [ ]:
from scripts import load_data_to_mongo

- Export Data to MongoDB

In [ ]:
data = df.to_dict(orient='index')

In [ ]:
load_data_to_mongo.load_to_mongo(data, 'clean')